In [1]:
#!/usr/bin/env python
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from math import factorial
from entropy import ent_kde, ent_knn

# Environmnent variables
dim = 2
nframes = 1000


In [2]:
def gen_sample(dist, dim, nframes, xmin, xmax, *args):
    """
    Generate a sample of a given distribution via Monte Carlo simulation
    """
    rng = np.random.default_rng(2021)
    sample = []

    while nframes > 0:
        dom = rng.uniform(xmin, xmax, dim)
        img = dist(dom, *args)
        ran = rng.uniform(0, 1)
        if img > ran:
           nframes -= 1
           sample.append(dom)
    
    return np.array(sample)
    

In [3]:
# 1. Normal distribution

def norm_dist(x, mean, cov):
    x_m = x - mean
    return (1. / (np.sqrt((2 * np.pi)**dim * np.linalg.det(cov))) *
        np.exp(-(np.linalg.solve(cov, x_m).T.dot(x_m)) / 2))


r1 = .1
r2 = .9
mean = np.zeros(dim)
cov = np.array([
    [1, r1],
    [r2, 1]]
)
xmin, xmax = -3, 3

X1 = gen_sample(norm_dist, dim, nframes, xmin, xmax, mean, cov).reshape(-1, 2)

norm_ent = 0.5 * np.log((2*np.pi*np.e)**dim * np.linalg.det(cov))
kde_norm = ent_kde(X1, bound1=xmin, bound2=xmax)
knn_norm = ent_knn(X1, n_neighbors=10)

norm_ent = round(norm_ent, 4)
kde_norm = round(kde_norm, 4)
knn_norm = round(knn_norm, 4)

print(f'Normal distribution with mean {mean} and variance\n{cov}')
print('Real entropy: ', norm_ent)
print('KDE entropy:  ', kde_norm)
print('KNN entropy:  ', knn_norm)


Normal distribution with mean [0. 0.] and variance
[[1.  0.1]
 [0.9 1. ]]
Real entropy:  2.7907
KDE entropy:   2.8472
KNN entropy:   2.6816


In [4]:
# 2. Logistic distribution

def log_dist(x, s, mu):
    sum_exp = np.sum(np.exp(-(x-mu) / s))
    #exp_sum = np.exp(np.sum(-(x-mu) / s))
    return (factorial(dim) / s.prod()) * sum_exp * (1 + sum_exp)**-(dim+1)


def log_ent():
    def A(p):
        if p == 1:
            return 1
        else:
            p_fact = factorial(p) / factorial(p-1)
            return (p_fact/p**2) + (p_fact/p)*A(p-1)

    return np.sum(np.log(s)) - np.log(factorial(dim)) + (dim+1)*A(dim)


# Location (mu) could take any value and scale (s) must be greater than 0
mu, s = np.zeros(dim) + 5, np.zeros(dim) + 3
xmin, xmax = -10, 30

X2 = gen_sample(log_dist, dim, nframes, xmin, xmax, s, mu).reshape(-1, 2)

log_ent = log_ent()
kde_log = ent_kde(X2, bound1=xmin, bound2=xmax)
knn_log = ent_knn(X2, n_neighbors=4)

log_ent = round(log_ent, 4)
kde_log = round(kde_log, 4)
knn_log = round(knn_log, 4)

print(f'Logistic distribution with location mu={mu} and scale s={s}')
print('Real entropy: ', log_ent)
print('KDE entropy:  ', kde_log)
print('KNN entropy:  ', knn_log)


Logistic distribution with location mu=[5. 5.] and scale s=[3. 3.]
Real entropy:  6.0041
KDE entropy:   6.5125
KNN entropy:   6.4222


In [5]:
# 3. Exponential distribution

def exp_dist(x, lamb):
    arg = [lamb + i - 1 for i in range(1, dim+1)]
    sum_exp = np.sum(np.exp(x))
    return np.prod(arg) * ((sum_exp-dim+1)**(-(lamb+dim))) * sum_exp


def exp_ent():
    arg = [lamb + i - 1 for i in range(1, dim+1)]
    arg_2 = [lamb + i for i in range(1, dim+1)]
    prods = (np.prod(arg)*np.sum(arg)) / ((np.sum(arg)-1)**2 * (np.sum(arg_2)-2))
    return np.sum(np.log(arg)) + (dim*np.log(lamb)) - (dim*lamb*prods)


lamb = 2
xmin, xmax = 0.1, 5

# Domain of the distribution must be positive
X3 = gen_sample(exp_dist, dim, nframes, xmin, xmax, lamb).reshape(-1, 2)

exp_ent = exp_ent()
kde_exp = ent_kde(X3, bound1=xmin, bound2=xmax)
knn_exp = ent_knn(X3, n_neighbors=10)

exp_ent = round(exp_ent, 4)
kde_exp = round(kde_exp, 4)
knn_exp = round(knn_exp, 4)

print(f'Exponential distribution with rate {lamb}')
print('Real entropy: ', exp_ent)
print('KDE entropy:  ', kde_exp)
print('KNN entropy:  ', knn_exp)


Exponential distribution with rate 2
Real entropy:  1.6781
KDE entropy:   0.6719
KNN entropy:   0.5111


In [6]:
# 4. Pareto distribution

def par_dist(x, alpha, sigma, mu):
    #mu = x - 0.2
    arg = np.array([(alpha + i - 1)/mu[i-1] for i in range(1, dim+1)])
    sum = np.sum([((x[i]-mu[i])/sigma[i]) for i in range(0, dim)])
    return np.prod([(arg * (1 + sum)**-(alpha+i)) for i in range(1, dim+1)])


def par_ent():
    arg = np.array([alpha + i - 1 for i in range(1, dim+1)])
    arg_2 = np.array([alpha + i - 2 for i in range(1, dim+1)])
    prods = (np.prod(arg)*np.sum(arg)) / (np.sum(arg)**2 * np.sum(arg_2))
    return -np.sum(np.log(arg/sigma)) + prods


alpha = 2
mu = np.zeros(dim) + 1
sigma = np.zeros(dim) + 1
xmin, xmax = 5, 150

X4 = gen_sample(par_dist, dim, nframes, xmin, xmax, alpha, sigma, mu).reshape(-1, 2)

par_ent = par_ent()
kde_par = ent_kde(X4, bound1=xmin, bound2=xmax)
knn_par = ent_knn(X4, n_neighbors=100)

par_ent = round(par_ent, 4)
kde_par = round(kde_par, 4)
knn_par = round(knn_par, 4)

print(f'Pareto distribution with scale {mu} and shape {alpha}')
print('Real entropy: ', par_ent)
print('KDE entropy:  ', kde_par)
print('KNN entropy:  ', knn_par)


KeyboardInterrupt: 

In [14]:
# 5. Manual distribution

from scipy.integrate import dblquad

def my_dist(x, y):
    return 1/64 * (x + y)
    #return 6/5*(x**2 + y)
    #return 6/7*(x + y)**2


def entropy(x, y):
    return -my_dist(x, y) * np.log(my_dist(x, y))


xmin, xmax = 0, 4

# Monte Carlo sample generation
rng = np.random.default_rng(2021)
sample = []

n = 1000

while n > 0:
    dom = rng.uniform(xmin, xmax, dim)
    img = my_dist(dom[0], dom[1])
    ran = rng.uniform(0, 1)
    if img > ran:
        n -= 1
        sample.append(dom)


X5 = np.array(sample)

x1, x2 = xmin, xmax
y1, y2 = lambda x: xmin, lambda x: xmax

dist_test, _ = dblquad(my_dist, x1, x2, y1, y2)
real_ent, _ = dblquad(entropy, x1, x2, y1, y2)

real_ent = max(real_ent, 0)
kde_my = ent_kde(X5, 0, 4)
knn_my = ent_knn(X5, 10)

real_ent = round(real_ent, 4)
kde_my = round(kde_my, 4)
knn_my = round(knn_my, 4)

print("Distribution test: ", dist_test)
print("Real entropy: ", real_ent)
print("KDE entropy:  ", kde_my)
print("KNN entropy:  ", knn_my)


Distribution test:  1.0
Real entropy:  2.6817
KDE entropy:   2.4425
KNN entropy:   2.6416


In [1]:
# Plots
sns.set_theme(style="darkgrid")
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(8, 8))

# Legend box style
props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)

sns.kdeplot(x=X1[:,0], y=X1[:,1], fill=True, ax=ax1)
ax1.set_title("Normal Distribution")
ax1.set_xlabel("")
ax1.set_ylabel("")
legend_1 = f'Real ent: {norm_ent}\nKDE ent: {kde_norm}\nKNN ent: {knn_norm}'
ax1.text(0.60, 0.95, legend_1, transform=ax1.transAxes, fontsize=9,
    verticalalignment='top', bbox=props)

sns.kdeplot(x=X2[:,0], y=X2[:,1], fill=True, ax=ax2)
ax2.set_title("Logistic Distribution")
ax2.set_xlabel("")
ax2.set_ylabel("")
legend_2 = f'Real ent: {log_ent}\nKDE ent: {kde_log}\nKNN ent: {knn_log}'
ax2.text(0.60, 0.95, legend_2, transform=ax2.transAxes, fontsize=9,
    verticalalignment='top', bbox=props)

sns.kdeplot(x=X3[:,0], y=X3[:,1], fill=True, ax=ax3)
ax3.set_title("Exponential Distribution")
ax3.set_xlabel("")
ax3.set_ylabel("")
legend_3 = f'Real ent: {exp_ent}\nKDE ent: {kde_exp}\nKNN ent: {knn_exp}' 
ax3.text(0.60, 0.95, legend_3, transform=ax3.transAxes, fontsize=9,
    verticalalignment='top', bbox=props)

sns.kdeplot(x=X5[:,0], y=X5[:,1], fill=True, ax=ax4)
ax4.set_title("Manual Distribution f(x,y)=1/64(x+y)")
ax4.set_xlabel("")
ax4.set_ylabel("")
legend_4 = f'Real ent: {real_ent}\nKDE ent: {kde_my}\nKNN ent: {knn_my}'
ax4.text(0.60, 0.95, legend_4, transform=ax4.transAxes, fontsize=9,
    verticalalignment='top', bbox=props)

plt.suptitle("Evaluación de los modelos KNN y KDE para obtener la entropía"
    "de distintas distribuciones de probabilidad")
plt.show()


NameError: name 'sns' is not defined